In [1]:
import numpy as np
import pandas as pd
import glob
import sys,os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from multiprocessing import Pool, Manager, cpu_count 
import pymp
import qgrid
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "jupyterlab"
import gezi
from gezi import tqdm, line
tqdm.pandas()
from IPython.display import display

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('precision', 5)

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
V = 21
V2 = 21
MARK = 'offline'
# MARK = 'online'

In [4]:
def gen_df(root, pattern='metrics'):
  dfs= Manager().list()
  pattern = f'{root}/*/*/{pattern}.csv'
  files = glob.glob(pattern)
  if not files:
    return None
  files = sorted(files, key=lambda x: os.path.getmtime(x))
  ps = min(len(files), cpu_count())
  with pymp.Parallel(ps) as p:
#     for i in tqdm(p.range(len(files)),desc='gen_df'):
    for i in p.range(len(files)):
      file = files[i]
      if not gezi.non_empty(file):
        continue
      df = pd.read_csv(file)
      v = int(os.path.basename(os.path.dirname(os.path.dirname(file))))
      if v < V:
        continue
#       if v < V2 and len(df) <  4:
#         continue
      model_name = os.path.basename(os.path.dirname(file))
      v_ = model_name.split('-')[0]
      if v_.isdigit():
        continue
      df['model'] = f'{v}-{model_name}'
      df['mtime'] = os.path.getmtime(file)
      df['ctime'] = os.path.getctime(file)
      df['step'] = [x + 1 for x in range(len(df))]
      dfs.append(df)
  df = pd.concat(list(dfs))
  df = df.reset_index()
  return df

In [5]:
def gen_history(root):
  dfs= Manager().list()
  pattern = f'{root}/{V2}/*/history.csv'
  files = glob.glob(pattern)
  if not files:
    return None
  files = sorted(files, key=lambda x: os.path.getmtime(x))
  ps = min(len(files), cpu_count())
  with pymp.Parallel(ps) as p:
#     for i in tqdm(p.range(len(files)),desc='gen_history', leave=False):
    for i in p.range(len(files)):
      file = files[i]
      if not gezi.non_empty(file):
        continue
      df = pd.read_csv(file)
      model_name = os.path.basename(os.path.dirname(file))
      df['model'] = model_name
      df['mtime'] = os.path.getmtime(file)
      df['ctime'] = os.path.getctime(file)
      df['step'] = [x + 1 for x in range(len(df))]
      dfs.append(df)
  df = pd.concat(list(dfs))
  df = df.reset_index()
  return df

In [6]:
root = f'../working/{MARK}'
froot = f'/home/tione/notebook/working/figs/{MARK}'
os.system(f'mkdir -p {froot}')

0

In [7]:
history = gen_history(root)

In [8]:
# history.groupby('model')['epoch'].max().reset_index()

In [9]:
# history

In [10]:
# history.head()

In [11]:
# keys = ['train_loss', 'val_loss', 'lr']
# figs = line(history, keys, x='step', color='model', smoothing=0.8, return_figs=True)
# for key, fig in zip(keys, figs):
#   fig.write_html(f'{froot}/history_{key}.html', auto_open=True)

In [12]:
# def show_loss():
#   return history.groupby(['step', 'model'])['val_loss'] \
#     .aggregate(np.mean).reset_index() \
#     .pivot('step', 'model', 'val_loss')

In [13]:
# show_loss().tail()

In [14]:
def show(key, action='score'):
  metric = f'{key}/{action}'
  res = None
  try:
    res = df.groupby(['step', 'model'])[metric] \
      .aggregate(np.mean).reset_index() \
      .pivot('step', 'model', metric)
    figs = line(df, metric, x='step', color='model', return_figs=True)
    html_file = f'{froot}/metrics_{metric.replace("/", "-")}.html'
    figs[0].write_html(html_file, auto_open=True)
  except Exception:
    pass
  return res

In [15]:
df = gen_df(root)

In [16]:
# df = df[df.model.str.contains('mid')]

In [17]:
def rank(key, action='score', num=30):
  metric = f'{key}/{action}'
  try:
    d = df[['model', 'step', metric]].sort_values([metric], ascending=False).reset_index(drop=True)
    display(d.head(num))
  except Exception:
    pass

In [18]:
gezi.set_pandas(precision=5)

In [19]:
rank('all')

,model,step,all/score
0,21-ensemble,2,0.72347
1,21-large,1,0.71688
2,21-large.docfreeze,2,0.71661
3,21-big,2,0.71650
4,21-mid.sfu,2,0.71600
5,21-mid2,2,0.71600
6,21-mid.nofresh,2,0.71590
7,21-mid.user,2,0.71590
8,21-mid.merge,2,0.71590
9,21-mid.docfreeze,1,0.71587


In [20]:
rank('test_b')

,model,step,test_b/score
0,21-ensemble,2,0.72289
1,21-large,1,0.71645
2,21-large.docfreeze,2,0.71573
3,21-big,2,0.71520
4,21-mid.docfreeze,1,0.71507
5,21-large2,2,0.71500
6,21-mid2,2,0.71490
7,21-mid.user,2,0.71480
8,21-mid.feedfreeze,2,0.71480
9,21-mid.sfu,2,0.71480


In [21]:
rank('test_a')

,model,step,test_a/score
0,21-ensemble,2,0.72406
1,21-big,2,0.71780
2,21-large.docfreeze,2,0.71748
3,21-mid.merge,2,0.71740
4,21-large,1,0.71730
5,21-mid.sfu,2,0.71720
6,21-mid.autoint,2,0.71710
7,21-mid.nofresh,2,0.71700
8,21-mid2,2,0.71690
9,21-mid.user,2,0.71690


In [22]:
rank('colddoc')

,model,step,colddoc/score
0,21-ensemble,2,0.72058
1,21-large.docfreeze,2,0.71563
2,21-big,2,0.71360
3,21-large,1,0.71301
4,21-mid.feeddocfreeze,2,0.71300
5,21-mid.sfu,2,0.71270
6,21-mid.autoint,2,0.71250
7,21-large2,2,0.71220
8,21-mid.user,2,0.71200
9,21-mid.fresh2,2,0.71180


In [23]:
rank('test_b_colddoc')

,model,step,test_b_colddoc/score
0,21-ensemble,2,0.72436
1,21-large.docfreeze,2,0.72029
2,21-large2,2,0.71690
3,21-mid.doc2,2,0.71630
4,21-mid.sfu,2,0.71620
5,21-big,2,0.71610
6,21-large,1,0.71599
7,21-mid.user2,2,0.71540
8,21-mid.merge,2,0.71530
9,21-mid.share.doc2,2,0.71500


In [24]:
rank('test_a')

,model,step,test_a/score
0,21-ensemble,2,0.72406
1,21-big,2,0.71780
2,21-large.docfreeze,2,0.71748
3,21-mid.merge,2,0.71740
4,21-large,1,0.71730
5,21-mid.sfu,2,0.71720
6,21-mid.autoint,2,0.71710
7,21-mid.nofresh,2,0.71700
8,21-mid2,2,0.71690
9,21-mid.user,2,0.71690


In [25]:
rank('hot')

,model,step,hot/score
0,21-ensemble,2,0.72457
1,21-large,1,0.71812
2,21-large.docfreeze,2,0.71804
3,21-big,2,0.71780
4,21-mid.sfu,2,0.71720
5,21-mid.user,2,0.71710
6,21-mid.nofresh,2,0.71710
7,21-mid.docfreeze,1,0.71705
8,21-mid2,2,0.71700
9,21-mid.merge,2,0.71700


In [26]:
rank('cold')

,model,step,cold/score
0,21-ensemble,2,0.69559
1,21-mid.doc2,2,0.69010
2,21-mid.glove,2,0.68960
3,21-mid2,2,0.68900
4,21-mid.user2,2,0.68830
5,21-mid.ori,2,0.68820
6,21-mid.stay,2,0.68800
7,21-mid.share,2,0.68700
8,21-mid.merge.32,2,0.68690
9,21-large2,2,0.68680


In [27]:
rank('hotdoc')

,model,step,hotdoc/score
0,21-ensemble,2,0.72431
1,21-large,1,0.71793
2,21-mid2,2,0.71750
3,21-big,2,0.71720
4,21-mid.docfreeze,1,0.71718
5,21-large.docfreeze,2,0.71714
6,21-mid.merge,2,0.71710
7,21-mid.sfu,2,0.71700
8,21-mid.nofresh,2,0.71700
9,21-mid.user,2,0.71670


In [28]:
rank('test_b_hotdoc')

,model,step,test_b_hotdoc/score
0,21-ensemble,2,0.72277
1,21-large,1,0.71672
2,21-mid.docfreeze,1,0.71581
3,21-mid2,2,0.71580
4,21-mid.feedfreeze,2,0.71540
5,21-large.docfreeze,2,0.71532
6,21-mid.nofresh,2,0.71490
7,21-mid.user,2,0.71490
8,21-mid.sfu,2,0.71490
9,21-large2,2,0.71490


In [29]:
rank('test_b_cold')

,model,step,test_b_cold/score
0,21-ensemble,2,0.69127
1,21-mid.ori,2,0.68840
2,21-mid.glove,2,0.68780
3,21-mid.ori.docfreeze,2,0.68640
4,21-mid.doc2,2,0.68540
5,21-mid.share,2,0.68410
6,21-mid.user2,2,0.68400
7,21-mid.merge.32,2,0.68370
8,21-mid2,2,0.68290
9,21-mid.fresh,2,0.68090


In [30]:
rank('test_b_hot')

,model,step,test_b_hot/score
0,21-ensemble,2,0.72411
1,21-large,1,0.71796
2,21-large.docfreeze,2,0.71734
3,21-big,2,0.71690
4,21-mid.docfreeze,1,0.71647
5,21-large2,2,0.71640
6,21-mid.sfu,2,0.71630
7,21-mid.nofresh,2,0.71630
8,21-mid2,2,0.71620
9,21-mid.user,2,0.71620


In [31]:
# df.groupby(['step', 'model'])['all/score'].max().reset_index().head()

In [32]:
show('all')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.71688,0.65372,0.67978,0.65329,0.67893,NaN,NaN,0.6798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6775,0.71587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6784,NaN,NaN,NaN,0.6780,0.6774,NaN,NaN,NaN,NaN,0.6790,NaN,0.6468,0.6447,0.6565,0.6553
2,0.7165,0.72347,NaN,NaN,0.71661,0.71004,0.71182,0.7155,0.7155,NaN,0.7091,0.7156,0.7132,0.7152,0.6988,0.711,0.7153,NaN,NaN,0.7139,0.7153,0.7142,0.7155,0.7106,0.7127,0.7159,0.707,0.7159,0.7108,0.7104,0.716,0.7134,0.7109,0.7151,0.5009,0.7155,0.7159,0.7147,0.716,0.6885,0.6916,0.6795,0.6807


In [33]:
show('test_b')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.71645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7152,0.72289,NaN,NaN,0.71573,0.70927,0.71068,0.715,0.7147,NaN,0.7085,0.7141,0.713,0.7146,0.6989,0.7116,0.7146,NaN,NaN,0.7124,0.7148,0.7129,0.7143,0.7103,0.7118,0.7143,0.7066,0.7147,0.7105,0.7099,0.7148,0.7128,0.7101,0.7138,0.5016,0.7145,0.7148,0.7142,0.7149,0.6861,0.6903,0.68,0.6806


In [34]:
show('colddoc')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.71301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7136,0.72058,NaN,NaN,0.71563,0.70759,0.70978,0.7122,0.7113,NaN,0.7073,0.7125,0.7079,0.7089,0.6924,0.7057,0.7117,NaN,NaN,0.713,0.7117,0.7068,0.7118,0.706,0.7052,0.7106,0.7,0.7113,0.7061,0.706,0.7127,0.7094,0.7098,0.71,0.4939,0.7117,0.712,0.7113,0.7108,0.6833,0.6897,0.6509,0.6521


In [35]:
show('test_b_colddoc')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.71599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7161,0.72436,NaN,NaN,0.72029,0.71196,0.71287,0.7169,0.7134,NaN,0.7113,0.7148,0.7122,0.7131,0.6979,0.7107,0.7163,NaN,NaN,0.7141,0.7149,0.7087,0.7136,0.7116,0.7072,0.7153,0.6999,0.7149,0.7093,0.7114,0.7162,0.7114,0.715,0.7139,0.4982,0.7141,0.7141,0.7154,0.713,0.6821,0.6908,0.6536,0.6567


In [36]:
show('hot')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.71812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7178,0.72457,NaN,NaN,0.71804,0.71144,0.71296,0.7166,0.7167,NaN,0.7102,0.7168,0.7143,0.7163,0.6999,0.7122,0.7163,NaN,NaN,0.7151,0.7165,0.7153,0.7168,0.7115,0.7139,0.717,0.7077,0.7171,0.7118,0.7113,0.7172,0.7145,0.7119,0.7162,0.5007,0.7167,0.7171,0.7158,0.717,0.6895,0.6924,0.68,0.6811


In [37]:
show('cold')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.6854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.68561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.684,0.69559,NaN,NaN,0.68046,0.67679,0.68207,0.6868,0.6841,NaN,0.6816,0.684,0.6831,0.6859,0.672,0.6809,0.6901,NaN,NaN,0.683,0.6847,0.6868,0.684,0.6896,0.6827,0.6865,0.6869,0.6846,0.6882,0.6864,0.6852,0.687,0.6849,0.688,0.5066,0.6861,0.6828,0.6883,0.689,0.6652,0.6719,0.6671,0.6701


In [38]:
show('hotdoc')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.71793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7172,0.72431,NaN,NaN,0.71714,0.71089,0.71229,0.7164,0.7163,NaN,0.7101,0.7167,0.7146,0.7163,0.7001,0.7121,0.7163,NaN,NaN,0.7146,0.7166,0.7158,0.7161,0.7115,0.7143,0.7171,0.7083,0.717,0.7121,0.7118,0.717,0.7142,0.7114,0.7163,0.5017,0.7166,0.7167,0.7154,0.7175,0.6898,0.6926,0.6837,0.6848


In [39]:
show('all', 'loss')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.94571,0.93613,0.94258,0.93716,0.94085,NaN,NaN,0.9425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9431,0.94587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9430,NaN,NaN,NaN,0.9421,0.9428,NaN,NaN,NaN,NaN,0.9434,NaN,0.9349,0.9343,0.9414,0.9411
2,0.9458,0.9463,NaN,NaN,0.94577,0.94546,0.94524,0.9458,0.9457,NaN,0.9452,0.9457,0.9457,0.9458,0.9409,0.9456,0.9458,NaN,NaN,0.9456,0.9457,0.9457,0.9458,0.9457,0.9458,0.9457,0.9449,0.9458,0.9456,0.9456,0.9457,0.9454,0.9453,0.9458,0.8548,0.9458,0.9458,0.9458,0.9457,0.9441,0.9442,0.9437,0.9439


In [40]:
show('test_a')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.7173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7178,0.72406,NaN,NaN,0.71748,0.71081,0.71296,0.7159,0.7162,NaN,0.7097,0.7171,0.7135,0.7157,0.6987,0.7104,0.716,NaN,NaN,0.7153,0.7158,0.7155,0.7167,0.7109,0.7136,0.7174,0.7073,0.717,0.7111,0.7108,0.7172,0.7141,0.7116,0.7165,0.5001,0.7165,0.7169,0.7152,0.7169,0.691,0.6929,0.679,0.6809


In [41]:
show('test_b_cold')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.67817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.67911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.6708,0.69127,NaN,NaN,0.67478,0.67191,0.67295,0.6788,0.6771,NaN,0.6804,0.6754,0.6798,0.6801,0.6686,0.6757,0.6854,NaN,NaN,0.6725,0.6802,0.6809,0.6752,0.6878,0.6754,0.6786,0.6837,0.6723,0.6884,0.6864,0.677,0.6841,0.6803,0.6796,0.5078,0.6791,0.6758,0.684,0.6829,0.6541,0.6678,0.6643,0.6677


In [42]:
show('test_b_hot')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.71796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7169,0.72411,NaN,NaN,0.71734,0.7108,0.71212,0.7164,0.7162,NaN,0.7096,0.7155,0.7142,0.716,0.7001,0.7129,0.7158,NaN,NaN,0.714,0.7161,0.7141,0.7158,0.7113,0.7132,0.7158,0.7075,0.7163,0.7114,0.7108,0.7163,0.7138,0.7113,0.7151,0.5015,0.7158,0.7162,0.7154,0.7162,0.6874,0.6911,0.6806,0.6811


In [43]:
show('test_b_hotdoc')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.71672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.7148,0.72277,NaN,NaN,0.71532,0.70926,0.71041,0.7149,0.7145,NaN,0.709,0.7143,0.7133,0.7149,0.6996,0.7115,0.7146,NaN,NaN,0.7126,0.7154,0.7139,0.714,0.7099,0.7123,0.7149,0.7075,0.7149,0.7109,0.7103,0.7149,0.7132,0.7097,0.7143,0.5023,0.7148,0.7149,0.714,0.7158,0.6866,0.6909,0.6835,0.6841


In [44]:
show('all', 'follow')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.77278,0.69119,0.72145,0.68512,0.72512,NaN,NaN,0.7275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.724,0.77387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7244,NaN,NaN,NaN,0.7193,0.7232,NaN,NaN,NaN,NaN,0.7244,NaN,0.6872,0.6946,0.6931,0.7006
2,0.7718,0.78173,NaN,NaN,0.77402,0.75990,0.76911,0.7713,0.7705,NaN,0.7677,0.7689,0.769,0.7684,0.7467,0.7676,0.7737,NaN,NaN,0.7698,0.7715,0.7722,0.7725,0.7624,0.7651,0.773,0.7612,0.7731,0.7628,0.7606,0.7717,0.7711,0.7648,0.772,0.5039,0.7701,0.7705,0.7668,0.7717,0.7374,0.7380,0.7298,0.7287


In [45]:
show('all', 'read_comment')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.68119,0.60934,0.64280,0.61224,0.64306,NaN,NaN,0.6458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6414,0.68159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6463,NaN,NaN,NaN,0.6429,0.6398,NaN,NaN,NaN,NaN,0.6426,NaN,0.5968,0.5884,0.6272,0.6256
2,0.6817,0.68733,NaN,NaN,0.68046,0.67534,0.67726,0.68,0.6803,NaN,0.6722,0.6801,0.6786,0.681,0.6636,0.6762,0.6806,NaN,NaN,0.6786,0.6801,0.6789,0.6809,0.6763,0.679,0.6816,0.6759,0.6807,0.6753,0.676,0.6811,0.6780,0.6762,0.6808,0.5012,0.681,0.6815,0.6815,0.6802,0.6591,0.6620,0.6517,0.6525


In [46]:
show('all', 'favorite')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.79184,0.74264,0.75781,0.74164,0.76450,NaN,NaN,0.7573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7493,0.79291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7515,NaN,NaN,NaN,0.7525,0.7507,NaN,NaN,NaN,NaN,0.7551,NaN,0.7386,0.7383,0.7392,0.7375
2,0.7963,0.80206,NaN,NaN,0.79496,0.78698,0.79383,0.7926,0.7934,NaN,0.788,0.7942,0.7913,0.7912,0.7777,0.7883,0.7904,NaN,NaN,0.7899,0.7949,0.7924,0.7913,0.7859,0.7901,0.7934,0.7807,0.7934,0.7856,0.7898,0.7933,0.7843,0.7860,0.7926,0.4944,0.7929,0.7904,0.7915,0.7931,0.7663,0.7686,0.7634,0.7634


In [47]:
show('all', 'like')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.67978,0.63696,0.65353,0.63689,0.64917,NaN,NaN,0.6436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6531,0.68025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6524,NaN,NaN,NaN,0.6521,0.6553,NaN,NaN,NaN,NaN,0.6505,NaN,0.6206,0.6222,0.6233,0.6205
2,0.6808,0.68632,NaN,NaN,0.68109,0.67643,0.67328,0.6798,0.6792,NaN,0.6761,0.6793,0.6781,0.6794,0.6654,0.6765,0.6819,NaN,NaN,0.6781,0.6788,0.6794,0.6798,0.6776,0.6774,0.6788,0.6712,0.6803,0.6776,0.6768,0.6796,0.6784,0.6765,0.6797,0.4998,0.6794,0.679,0.6789,0.6806,0.6511,0.6547,0.6367,0.6368


In [48]:
show('all', 'comment')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.67489,0.58504,0.61963,0.57856,0.60830,NaN,NaN,0.6196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5976,0.65764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6023,NaN,NaN,NaN,0.6092,0.6001,NaN,NaN,NaN,NaN,0.6218,NaN,0.5766,0.5823,0.5787,0.5774
2,0.6607,0.68076,NaN,NaN,0.66674,0.65491,0.66168,0.6655,0.6643,NaN,0.6463,0.668,0.6561,0.6655,0.6443,0.6538,0.6549,NaN,NaN,0.666,0.6628,0.6573,0.6628,0.6521,0.6549,0.6652,0.6475,0.6667,0.6493,0.6468,0.6653,0.6655,0.6550,0.6571,0.5078,0.664,0.6717,0.6612,0.6651,0.6271,0.6307,0.6185,0.6269


In [49]:
show('all', 'follow')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.77278,0.69119,0.72145,0.68512,0.72512,NaN,NaN,0.7275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.724,0.77387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7244,NaN,NaN,NaN,0.7193,0.7232,NaN,NaN,NaN,NaN,0.7244,NaN,0.6872,0.6946,0.6931,0.7006
2,0.7718,0.78173,NaN,NaN,0.77402,0.75990,0.76911,0.7713,0.7705,NaN,0.7677,0.7689,0.769,0.7684,0.7467,0.7676,0.7737,NaN,NaN,0.7698,0.7715,0.7722,0.7725,0.7624,0.7651,0.773,0.7612,0.7731,0.7628,0.7606,0.7717,0.7711,0.7648,0.772,0.5039,0.7701,0.7705,0.7668,0.7717,0.7374,0.7380,0.7298,0.7287


In [50]:
show('all', 'click_avatar')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.77263,0.71393,0.74230,0.71216,0.74197,NaN,NaN,0.7458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7419,0.7729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7378,NaN,NaN,NaN,0.7392,0.7413,NaN,NaN,NaN,NaN,0.7410,NaN,0.7227,0.7163,0.7204,0.7185
2,0.7737,0.77817,NaN,NaN,0.77343,0.76778,0.76856,0.7724,0.7731,NaN,0.7693,0.772,0.7703,0.7718,0.7576,0.7675,0.7729,NaN,NaN,0.7703,0.7723,0.771,0.7723,0.7674,0.7707,0.7722,0.7637,0.7712,0.7708,0.7689,0.7727,0.7708,0.7700,0.7721,0.4995,0.772,0.771,0.7712,0.7732,0.7464,0.7490,0.7401,0.7406


In [51]:
show('all', 'forward')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.77054,0.70339,0.72185,0.70349,0.72450,NaN,NaN,0.7275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7278,0.76889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7227,NaN,NaN,NaN,0.7272,0.7247,NaN,NaN,NaN,NaN,0.7223,NaN,0.7113,0.7126,0.7044,0.7030
2,0.7694,0.77603,NaN,NaN,0.76826,0.76248,0.76302,0.7675,0.7682,NaN,0.7612,0.7696,0.7658,0.7664,0.7501,0.7638,0.7666,NaN,NaN,0.7655,0.7681,0.7674,0.7676,0.765,0.7659,0.7674,0.757,0.7667,0.7673,0.7652,0.7687,0.7648,0.7608,0.7687,0.5025,0.7685,0.7689,0.7669,0.7686,0.7378,0.7431,0.7253,0.7285


In [52]:
show('loss_all', 'mean')

model,21-big,21-ensemble,21-large,21-large.day,21-large.docfreeze,21-large.docfreeze.day,21-large.docfreeze2,21-large2,21-large3,21-large4,21-mid.allfreeze,21-mid.autoint,21-mid.cross,21-mid.cross2,21-mid.cross3,21-mid.day,21-mid.doc2,21-mid.doc3,21-mid.docfreeze,21-mid.feeddocfreeze,21-mid.feedfreeze,21-mid.fresh,21-mid.fresh2,21-mid.glove,21-mid.maskdoc,21-mid.merge,21-mid.merge.32,21-mid.nofresh,21-mid.ori,21-mid.ori.docfreeze,21-mid.sfu,21-mid.share,21-mid.share.doc2,21-mid.stay,21-mid.todays,21-mid.umin3,21-mid.user,21-mid.user2,21-mid2,23-tiny.glove,23-tiny.ori,23-tiny.scratch,23-tiny.scratch2
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.96598,0.95975,0.96400,0.96018,0.96295,NaN,NaN,0.964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9643,0.96606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9642,NaN,NaN,NaN,0.9637,0.9641,NaN,NaN,NaN,NaN,0.9644,NaN,0.9592,0.9589,0.9630,0.9628
2,0.9661,0.96636,NaN,NaN,0.96603,0.96578,0.96566,0.966,0.9659,NaN,0.9657,0.966,0.966,0.966,0.9631,0.9658,0.966,NaN,NaN,0.9659,0.966,0.966,0.966,0.966,0.966,0.966,0.9655,0.9660,0.9659,0.9659,0.966,0.9658,0.9657,0.966,0.9174,0.966,0.966,0.9660,0.966,0.9647,0.9648,0.9646,0.9647


In [53]:
# abc

In [54]:
ACTIONS = [
  'read_comment',
  'like',
  'click_avatar',
  'forward',
  'favorite',
  'comment',
  'follow'
]

In [55]:
for action in ['score'] + ACTIONS:
  key = f'all/{action}'
  display(df.groupby('model')[key].max().reset_index().sort_values(key, ascending=False).head(10))

,model,all/score
1,21-ensemble,0.72347
2,21-large,0.71688
4,21-large.docfreeze,0.71661
0,21-big,0.71650
38,21-mid2,0.71600
30,21-mid.sfu,0.71600
36,21-mid.user,0.71590
27,21-mid.nofresh,0.71590
25,21-mid.merge,0.71590
18,21-mid.docfreeze,0.71587


,model,all/read_comment
1,21-ensemble,0.68733
0,21-big,0.68170
25,21-mid.merge,0.68160
18,21-mid.docfreeze,0.68159
37,21-mid.user2,0.68150
36,21-mid.user,0.68150
2,21-large,0.68119
30,21-mid.sfu,0.68110
13,21-mid.cross2,0.68100
35,21-mid.umin3,0.68100


,model,all/like
1,21-ensemble,0.68632
16,21-mid.doc2,0.68190
4,21-large.docfreeze,0.68109
0,21-big,0.68080
38,21-mid2,0.68060
27,21-mid.nofresh,0.68030
18,21-mid.docfreeze,0.68025
7,21-large2,0.67980
22,21-mid.fresh2,0.67980
2,21-large,0.67978


,model,all/click_avatar
1,21-ensemble,0.77817
0,21-big,0.77370
4,21-large.docfreeze,0.77343
38,21-mid2,0.77320
8,21-large3,0.77310
18,21-mid.docfreeze,0.77290
16,21-mid.doc2,0.77290
30,21-mid.sfu,0.77270
2,21-large,0.77263
7,21-large2,0.77240


,model,all/forward
1,21-ensemble,0.77603
2,21-large,0.77054
11,21-mid.autoint,0.76960
0,21-big,0.76940
36,21-mid.user,0.76890
18,21-mid.docfreeze,0.76889
33,21-mid.stay,0.76870
30,21-mid.sfu,0.76870
38,21-mid2,0.76860
35,21-mid.umin3,0.76850


,model,all/favorite
1,21-ensemble,0.80206
0,21-big,0.79630
4,21-large.docfreeze,0.79496
20,21-mid.feedfreeze,0.79490
11,21-mid.autoint,0.79420
6,21-large.docfreeze2,0.79383
8,21-large3,0.79340
27,21-mid.nofresh,0.79340
25,21-mid.merge,0.79340
30,21-mid.sfu,0.79330


,model,all/comment
1,21-ensemble,0.68076
2,21-large,0.67489
36,21-mid.user,0.67170
11,21-mid.autoint,0.66800
4,21-large.docfreeze,0.66674
27,21-mid.nofresh,0.66670
19,21-mid.feeddocfreeze,0.66600
7,21-large2,0.66550
31,21-mid.share,0.66550
13,21-mid.cross2,0.66550


,model,all/follow
1,21-ensemble,0.78173
4,21-large.docfreeze,0.77402
18,21-mid.docfreeze,0.77387
16,21-mid.doc2,0.77370
27,21-mid.nofresh,0.77310
25,21-mid.merge,0.77300
2,21-large,0.77278
22,21-mid.fresh2,0.77250
21,21-mid.fresh,0.77220
33,21-mid.stay,0.77200


In [56]:
for prefix in ['first', 'cold', 'hot', 'colddoc', 'hotdoc', 'user0', 'user1']:
  key = f'{prefix}/score'
  if key in df.columns:
    display(df.groupby('model')[key].max().reset_index().sort_values(key, ascending=False).head(10))

,model,cold/score
1,21-ensemble,0.69559
16,21-mid.doc2,0.69010
23,21-mid.glove,0.68960
38,21-mid2,0.68900
37,21-mid.user2,0.68830
28,21-mid.ori,0.68820
33,21-mid.stay,0.68800
31,21-mid.share,0.68700
26,21-mid.merge.32,0.68690
7,21-large2,0.68680


,model,hot/score
1,21-ensemble,0.72457
2,21-large,0.71812
4,21-large.docfreeze,0.71804
0,21-big,0.71780
30,21-mid.sfu,0.71720
27,21-mid.nofresh,0.71710
36,21-mid.user,0.71710
18,21-mid.docfreeze,0.71705
38,21-mid2,0.71700
25,21-mid.merge,0.71700


,model,colddoc/score
1,21-ensemble,0.72058
4,21-large.docfreeze,0.71563
0,21-big,0.71360
2,21-large,0.71301
19,21-mid.feeddocfreeze,0.71300
30,21-mid.sfu,0.71270
11,21-mid.autoint,0.71250
7,21-large2,0.71220
36,21-mid.user,0.71200
22,21-mid.fresh2,0.71180


,model,hotdoc/score
1,21-ensemble,0.72431
2,21-large,0.71793
38,21-mid2,0.71750
0,21-big,0.71720
18,21-mid.docfreeze,0.71718
4,21-large.docfreeze,0.71714
25,21-mid.merge,0.71710
30,21-mid.sfu,0.71700
27,21-mid.nofresh,0.71700
11,21-mid.autoint,0.71670


In [57]:
# models = [(x, df[df.model==x]['ctime'][0]) for x in set(df.model)]
# models.sort(key=lambda x: -x[-1])
# models_ = [x[0] for x in models[:30]]
# models_

In [58]:
# metrics = ['all/score']
# df_ = df[df.model.isin(models_)]
# df_[['model', 'step'] + metrics].tail()

In [59]:
# line(df_, metrics, x='step', color='model')